<a href="https://colab.research.google.com/github/sanu0711/langChain-experiments/blob/main/RAG_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf langchain langchain-community chromadb langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 23.4 MB/s eta 0:00:00

In [17]:
!pip install langchain_chroma

# Simple RAG (Single PDF file)

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_community.vectorstores import Chroma    # old version
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from google.colab import userdata





## Loading PDF Data

In [3]:
document_path = "/content/Abhishek_Yadav.pdf"

loader = PyMuPDFLoader(document_path)
data = loader.load()
print(data)
# print(len(data))
# print(data[0].page_content)
# print(data[0].metadata)

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': "LaTeX with 'moderncv' package", 'creationdate': '2024-11-21T17:03:42+00:00', 'source': '/content/Abhishek_Yadav.pdf', 'file_path': '/content/Abhishek_Yadav.pdf', 'total_pages': 2, 'format': 'PDF 1.5', 'title': 'Abhishek Yadav –', 'author': 'Abhishek Yadav', 'subject': 'Résumé of Abhishek Yadav', 'keywords': 'Abhishek Yadav, curriculum vitæ, résumé', 'moddate': '2024-11-21T17:03:42+00:00', 'trapped': '', 'modDate': 'D:20241121170342Z', 'creationDate': 'D:20241121170342Z', 'page': 0}, page_content='Abhishek Yadav\nJankipuram Extension – Lucknow, Uttar Pradesh, India\nI +91 7985459061\n•\n# sanuabhi.0711@gmail.com\n\x82 sanu0711.pythonanywhere.com\n•\nð abhishek-yadav-034825220\n•\n§ sanu0711\nEducation\nRajiv Gandhi Institute of Petroleum Technology (RGIPT)\nJais, UP, India\nB.Tech. in Information Technology, CGPA: 9.08/10\n2021–Present\nRameshwaram International Academy\nLucknow, India\nSenior Secondary (XII), Science, Percen

In [ ]:
# from langchain.document_loaders import TextLoader # to load text file

# text_loader = TextLoader("/content/data.txt")
# documents = text_loader.load()


##  Split the Data into Chunks

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
  )
chunks = text_splitter.split_documents(data)
print(f"Split documents into {len(chunks)} chunks.")
document = chunks[0]
print(document.metadata)
print(document.page_content)

Split documents into 11 chunks.
{'producer': 'pdfTeX-1.40.25', 'creator': "LaTeX with 'moderncv' package", 'creationdate': '2024-11-21T17:03:42+00:00', 'source': '/content/Abhishek_Yadav.pdf', 'file_path': '/content/Abhishek_Yadav.pdf', 'total_pages': 2, 'format': 'PDF 1.5', 'title': 'Abhishek Yadav –', 'author': 'Abhishek Yadav', 'subject': 'Résumé of Abhishek Yadav', 'keywords': 'Abhishek Yadav, curriculum vitæ, résumé', 'moddate': '2024-11-21T17:03:42+00:00', 'trapped': '', 'modDate': 'D:20241121170342Z', 'creationDate': 'D:20241121170342Z', 'page': 0, 'start_index': 0}
Abhishek Yadav
Jankipuram Extension – Lucknow, Uttar Pradesh, India
I +91 7985459061
•
# sanuabhi.0711@gmail.com
 sanu0711.pythonanywhere.com
•
ð abhishek-yadav-034825220
•
§ sanu0711
Education
Rajiv Gandhi Institute of Petroleum Technology (RGIPT)
Jais, UP, India
B.Tech. in Information Technology, CGPA: 9.08/10
2021–Present
Rameshwaram International Academy
Lucknow, India
Senior Secondary (XII), Science, Percentage

## Create Vector Embeddings and Store Vectors in a Vector Database

In [6]:
chroma_path = "/content/chroma_db"
embeddings = OpenAIEmbeddings(
    api_key=userdata.get('OPENAI_API_KEY'),
    model="text-embedding-3-small"
)

vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=chroma_path)
# vectorstore.persist()  No need to write automatically save data on location in new updated version

## Retrieve Relevant Documents from Vector DB

In [7]:
query = "What are the key points in the document?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

In [8]:
print("length of retrieved docs: ", len(retrieved_docs))
print("------------------------------------------")
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].page_content)
  print("------------------------------------------")

length of retrieved docs:  3
------------------------------------------
- This helps businesses achieve a success rate of more than 90% in their projects.
Object Detection with YOLO:
- Engineered a Python-based object detection system using YOLO and OpenCV, achieving over 90% accuracy.
AI Document Search System:
- Designed a document search application using Llama Index and OpenAI GPT for natural language querying.
- Supported persistent indexing and customizable similarity thresholds, improving search accuracy by 25%.
Technical Skills
------------------------------------------
- This helps businesses achieve a success rate of more than 90% in their projects.
Object Detection with YOLO:
- Engineered a Python-based object detection system using YOLO and OpenCV, achieving over 90% accuracy.
AI Document Search System:
- Designed a document search application using Llama Index and OpenAI GPT for natural language querying.
- Supported persistent indexing and customizable similarity threshol

In [9]:
#  fetching from stored db once data is saved
chroma_path = "/content/chroma_db"
vector_data = Chroma(persist_directory=chroma_path, embedding_function=embeddings)

query = "tell me about abhishek yadav?"
retrieved_docs = vector_data.similarity_search(query, k=3)
print("length of retrieved docs: ", len(retrieved_docs))
print("------------------------------------------")
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].page_content)
  print("------------------------------------------")


length of retrieved docs:  3
------------------------------------------
Abhishek Yadav
Jankipuram Extension – Lucknow, Uttar Pradesh, India
I +91 7985459061
•
# sanuabhi.0711@gmail.com
 sanu0711.pythonanywhere.com
•
ð abhishek-yadav-034825220
•
§ sanu0711
Education
Rajiv Gandhi Institute of Petroleum Technology (RGIPT)
Jais, UP, India
B.Tech. in Information Technology, CGPA: 9.08/10
2021–Present
Rameshwaram International Academy
Lucknow, India
Senior Secondary (XII), Science, Percentage: 88.4%
2019–2020
Rameshwaram International Academy
Lucknow, India
------------------------------------------
Abhishek Yadav
Jankipuram Extension – Lucknow, Uttar Pradesh, India
I +91 7985459061
•
# sanuabhi.0711@gmail.com
 sanu0711.pythonanywhere.com
•
ð abhishek-yadav-034825220
•
§ sanu0711
Education
Rajiv Gandhi Institute of Petroleum Technology (RGIPT)
Jais, UP, India
B.Tech. in Information Technology, CGPA: 9.08/10
2021–Present
Rameshwaram International Academy
Lucknow, India
Senior Secondary (XII

In [10]:
#  other retriving methods similarity_search_with_score , similarity_search_with_relevance_scores,
#  max_marginal_relevance_search, max_marginal_relevance_search_with_relevance_scores,

similarity_search_with_score_data = vectorstore.similarity_search_with_score(query, k=2)
print(similarity_search_with_score_data[0])
print("reverence score: ", similarity_search_with_score_data[0][1])


(Document(id='ecda12fb-8641-48bc-ac9d-c7fdb319e452', metadata={'author': 'Abhishek Yadav', 'creationDate': 'D:20241121170342Z', 'creationdate': '2024-11-21T17:03:42+00:00', 'creator': "LaTeX with 'moderncv' package", 'file_path': '/content/Abhishek_Yadav.pdf', 'format': 'PDF 1.5', 'keywords': 'Abhishek Yadav, curriculum vitæ, résumé', 'modDate': 'D:20241121170342Z', 'moddate': '2024-11-21T17:03:42+00:00', 'page': 0, 'producer': 'pdfTeX-1.40.25', 'source': '/content/Abhishek_Yadav.pdf', 'start_index': 0, 'subject': 'Résumé of Abhishek Yadav', 'title': 'Abhishek Yadav –', 'total_pages': 2, 'trapped': ''}, page_content='Abhishek Yadav\nJankipuram Extension – Lucknow, Uttar Pradesh, India\nI +91 7985459061\n•\n# sanuabhi.0711@gmail.com\n\x82 sanu0711.pythonanywhere.com\n•\nð abhishek-yadav-034825220\n•\n§ sanu0711\nEducation\nRajiv Gandhi Institute of Petroleum Technology (RGIPT)\nJais, UP, India\nB.Tech. in Information Technology, CGPA: 9.08/10\n2021–Present\nRameshwaram International Aca

## Generate Responses Using LLMs

In [19]:
# context = "\n\n".join([
#     f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}"
#     for doc in retrieved_docs
# ])
# print(context)  # we can use this content in custom prompt

# Direct method using RetrievalQA

llm = ChatOpenAI(
    api_key=userdata.get('OPENAI_API_KEY'),
    model="gpt-4"
)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # Change this to "map_reduce", "refine", or "map_rerank" as needed
    retriever=vector_data.as_retriever(),
    return_source_documents=True
)

query = "Personal details of abhishek yadav?"
result = chain.invoke({"query": query}, return_only_outputs=True)


In [20]:
result

{'result': 'Abhishek Yadav is located in Jankipuram Extension, Lucknow, Uttar Pradesh, India. His contact number is +91 7985459061. His email address is sanuabhi.0711@gmail.com and his website is sanu0711.pythonanywhere.com. He is also available on a platform with the id abhishek-yadav-034825220 and his online handle on something else is sanu0711.',
 'source_documents': [Document(id='ecda12fb-8641-48bc-ac9d-c7fdb319e452', metadata={'author': 'Abhishek Yadav', 'creationDate': 'D:20241121170342Z', 'creationdate': '2024-11-21T17:03:42+00:00', 'creator': "LaTeX with 'moderncv' package", 'file_path': '/content/Abhishek_Yadav.pdf', 'format': 'PDF 1.5', 'keywords': 'Abhishek Yadav, curriculum vitæ, résumé', 'modDate': 'D:20241121170342Z', 'moddate': '2024-11-21T17:03:42+00:00', 'page': 0, 'producer': 'pdfTeX-1.40.25', 'source': '/content/Abhishek_Yadav.pdf', 'start_index': 0, 'subject': 'Résumé of Abhishek Yadav', 'title': 'Abhishek Yadav –', 'total_pages': 2, 'trapped': ''}, page_content='Ab

In [21]:
result["result"]

'Abhishek Yadav is located in Jankipuram Extension, Lucknow, Uttar Pradesh, India. His contact number is +91 7985459061. His email address is sanuabhi.0711@gmail.com and his website is sanu0711.pythonanywhere.com. He is also available on a platform with the id abhishek-yadav-034825220 and his online handle on something else is sanu0711.'

In [14]:
retriever = vector_data.as_retriever(search_kwargs={"k": 3})
print(retriever)


tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d0402f1b910> search_kwargs={'k': 3}


# Directory (*pdf, *txt, *docx)

In [49]:
!pip install pypdf docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=2acf8749b7a37f03591d8ed211b4a61bb189a28c96d576a8703d592bc46ad8d8
  Stored in directory: /root/.cache/pip/wheels/0f/0e/7a/3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


In [42]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

dir_path = "/content/data/"
loader = PyPDFDirectoryLoader(dir_path)
documents = loader.load()
print("Loaded", len(documents), "documents")


Loaded 13 documents


In [64]:
import os
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    if os.path.isfile(file_path): # ensure file not folder
        print(file_path)
    # print(file_path)


/content/data/IG-Blog.docx
/content/data/Abhishek_Yadav.pdf
/content/data/IG-VOTE.docx
/content/data/text_file.txt
/content/data/mc.pdf


In [50]:
import os
from langchain_community.document_loaders import (
    PyPDFDirectoryLoader,
    PyMuPDFLoader,
    UnstructuredFileLoader,
    TextLoader,
    CSVLoader,
    Docx2txtLoader
)

dir_path = "/content/data/"
documents = []

# Supported file loaders
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)

    if filename.endswith(".pdf"):
        loader = PyMuPDFLoader(file_path)  # PDF loader
    elif filename.endswith(".txt"):
        loader = TextLoader(file_path)  # Text file loader
    elif filename.endswith(".csv"):
        loader = CSVLoader(file_path)  # CSV file loader
    elif filename.endswith(".docx") or filename.endswith(".doc"):
        loader = Docx2txtLoader(file_path)  # Word file loader
    else:
        print(f"Skipping unsupported file: {filename}")
        continue  # Skip unsupported files

    # Load and extend documents
    documents.extend(loader.load())

print("Loaded", len(documents), "documents")


Skipping unsupported file: .ipynb_checkpoints
Loaded 16 documents


In [63]:
# for i in documents:
#     print(i.metadata["source"])

# Understanding chain_type in RetrievalQA

## 1. "stuff" (Default)

* **How it works:** Retrieves all relevant documents, concatenates their content into a single string, and passes it to the LLM as context.

* **Best for**: Small document sets or when the LLM's token limit allows processing all retrieved documents at once.

* **Limitation**: If many documents are retrieved, it may exceed the model's token limit.

## 2. "map_reduce"

* **How it works:** The LLM first summarizes each document individually.
Then, it combines these summaries into a final answer.

* **Best for:** Large document sets where concatenation would exceed token limits.

* **Limitation**: Less direct context available in each LLM call.

## 3. "refine"

* **How it works:**
The LLM starts with an initial answer from the first document.
Then, it iteratively refines the answer using each subsequent document.

* **Best for:** When detailed information from multiple sources needs to be integrated sequentially.

* **Limitation:** Errors in early stages might persist in refinement.

## 4. "map_rerank"

* **How it works:**
The LLM generates an answer for each retrieved document.
Then, it ranks the answers based on relevance or confidence.

* **Best for:** Cases where document importance varies, and the best response needs to be selected.

* **Limitation**: Can be expensive due to multiple LLM calls